#Answer 2

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


**install required module (will take 45 second)**

In [4]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=c7dd6b26b9b319d1f49f56062b6ad06893f3570145418bac57d09fd29b51f766
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.6 MB/s eta 0:00:00


#Naampadam Dataset Loading

The _Naampadam_ Dataset is a large dataset for Named Entity Recognition in 11 Indian languages.  _Naampadam_ means "named entity" in Sanskrit.

**will take approx 15 second**

In [32]:
# download the Naampadam (Indic NER) hindi dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/985787 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/867 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13460 [00:00<?, ? examples/s]

In [4]:
# print how the Dataset looks like
raw_datasets['train']

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 985787
})

In [ ]:
data = raw_datasets['train']
data

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 985787
})

**make pandas dataframe (will take approx 2 minute)**

In [ ]:
# import pandas as pd
# df = pd.DataFrame(data)

In [ ]:
# print(df.head())

**Filter test Sentences for training(will take approx 30 sec)**

In [ ]:
# from collections import Counter
# percentage = 0.75
# filtered_df = df[df.apply(lambda row: Counter(row['ner_tags']).most_common()[0][1] * percentage <= sum(x[1] for x in Counter(row['ner_tags']).most_common() if x[0] != 0), axis=1)]


# filtered_df



In [ ]:
from datasets import Dataset
filtered_raw_data = Dataset.from_pandas(filtered_df)


In [ ]:
filtered_raw_data = filtered_raw_data.remove_columns('__index_level_0__')

In [ ]:
filtered_raw_data

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 84497
})

In [ ]:
raw_datasets["train"]

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 985787
})

In [ ]:
text_column_name = "tokens"
label_column_name = "ner_tags"

# Training an IndicNER Model with the dataset

download a pre-trained model and fine-tune it for the task of NER. I have have to use the `AutoModelForTokenClassification` class to fine-tune the model

**Load Pre-trained Model**

In [ ]:

check_output_dir = "/content/drive/My Drive/IndicNER_3epoch_batch8"


In [ ]:

# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")


In [ ]:
# **Run the next cell  to use a GPU**

model=model.to("cuda")

**Tokenize all texts and align the labels with them**

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=128,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

**preparing train dataset**

In [ ]:

from datasets import Dataset
train_dataset = raw_datasets["train"]
# from train data sampling 1 lakhs sentences
samples_train_data = train_dataset.shuffle(seed=42).select(range(20000))

# samples_train_data = filtered_raw_data
samples_train_data = samples_train_data.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",

)



**preparing validation data**

In [ ]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
      tokenize_and_align_labels,
      batched=True,
      num_proc=4,
      load_from_cache_file=True,
      desc="Running tokenizer on Validation dataset",
)


**Set Training Arguments**

In [ ]:
from transformers import TrainingArguments ,Trainer

args=TrainingArguments(
    output_dir=check_output_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_steps=4000,
    )


In [ ]:

trainer = Trainer(
    model=model,
    train_dataset=samples_train_data,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=args,
)



##Training

```
trainer.args
```

In [ ]:

train_result = trainer.train()



Step,Training Loss
500,0.371400
1000,0.174000
1500,0.177200
2000,0.181500
2500,0.170900
3000,0.127400
3500,0.117500
4000,0.115300
4500,0.118900
5000,0.129700


**Saving finetuned model on drive**

In [ ]:

try:
    trainer.save_model("/content/drive/My Drive/Indic_Ner_3epoch_batch8")
    print("Model saved successfully.")
except Exception as e:
    print("Error:", e)



Model saved successfully.


In [ ]:
print("moin")

##Fine-Tuning IndicBERT

### Let us now load the Pre-trained model

In [ ]:

# Define the output directory in your Google Drive
check_output_dir = "/content/drive/My Drive/checkpoint_Bert_batch8"


In [ ]:

from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=7, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=7 )


Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

model=model.to("cuda")


**Tokenize all texts and align the labels with them**

In [ ]:
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=128,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


**prepare training data**

```
from datasets import Dataset
train_dataset = raw_datasets["train"]

# from train data sampling 1 lakhs sentences
# samples_train_data = train_dataset.shuffle(seed=42).select(range(100000))
samples_train_data = filtered_raw_data
samples_train_data = samples_train_data.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)
```

**prepare validation data**

```
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)
```

## Set Training Arguments

In [ ]:

args=TrainingArguments(
    output_dir=check_output_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
     save_steps=4000 ,)


In [ ]:

# Initialize the trainer
trainer = Trainer(
    model=model,
    train_dataset=samples_train_data,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=args,
)


## Train the model

In [ ]:

train_result = trainer.train()


Step,Training Loss
500,0.671700
1000,0.504700
1500,0.454400
2000,0.422600
2500,0.387700
3000,0.339300
3500,0.318500
4000,0.307900
4500,0.309700
5000,0.316700


Step,Training Loss
500,0.671700
1000,0.504700
1500,0.454400
2000,0.422600
2500,0.387700
3000,0.339300
3500,0.318500
4000,0.307900
4500,0.309700
5000,0.316700


**saving model**

In [ ]:
try:
    # Save the trained model to your Google Drive
    trainer.save_model("/content/drive/My Drive/IndicBERT-3epoch_batch8")
    print("Model saved successfully.")
except Exception as e:
    print("Error:", e)



Model saved successfully.


# Testing & Prediction

**Storing true tags**

**function to make prediction**

In [20]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt',padding=True,truncation=True,max_length=128)

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

**function to remove tag having imbalance length**

In [21]:
def remove_imbalance_sentences(true_tags,predicted_tags):
      true_balance_tags = []
      predicted_balance_tags = []
      print(type(predicted_tags), type(true_tags))
      for i in range(len(true_tags)):
          if(len(true_tags[i])==len(predicted_tags[i])):
                true_balance_tags.append(true_tags[i])
                predicted_balance_tags.append(predicted_tags[i])
      return true_balance_tags,predicted_balance_tags




**function for calculating various measures**

In [22]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_scores(true_tags, predicted_tags):
    # remove tags having  different length
    true_tags,predicted_tags = remove_imbalance_sentences(true_tags,predicted_tags)

    true_tags_flat = np.concatenate(true_tags)
    predicted_tags_flat = np.concatenate(predicted_tags)

    # Find unique classes
    classes = np.unique(true_tags_flat)

    # Initialize lists to store precision, recall, and F1 scores for each class
    precisions = []
    recalls = []
    f1_scores = []

    # Calculate precision, recall, and F1 score for each class
    for class_id in classes:
        true_class = np.array(true_tags_flat) == class_id
        pred_class = np.array(predicted_tags_flat) == class_id

        precision = precision_score(true_class, pred_class, zero_division=1)
        recall = recall_score(true_class, pred_class, zero_division=1)
        f1 = f1_score(true_class, pred_class, zero_division=1)

        # Append scores to respective lists
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f"Class {label_to_tags[int_id_to_label[class_id]]}:")
        print(f"  Precision: {precision}")
        print(f"  Recall: {recall}")
        print(f"  F1 Score: {f1}")

    # Calculate macro F1 score
    macro_f1 = sum(f1_scores) / len(f1_scores)

    print("----------------------------")
    print(f"Macro F1 Score: {macro_f1}")

In [ ]:
# print(raw_datasets['test']['tokens'])

**prediction funtion for whole dataset**

In [42]:
data=raw_datasets['test']['tokens']

In [85]:
import torch
def  prediction_test(path,data=raw_datasets['test']['tokens'],flag=0):
      model,tokenizer  =  load_model(path)
      predicted_tags = []
      for input_text in data:
          if flag == 0:
             sentence = " ".join(input_text)
          else:
            sentence = input_text
          prediction = get_predictions(sentence=sentence,
                                    tokenizer=tokenizer,
                                    model=model
                                    )
          last_part = path.split('/')[-1]
          if last_part != "Indic_Ner_3epoch_batch_8" and last_part!= "Indic_Ner_3epoch_batch_16" and last_part!="filtrered_Indic_NEr_1lakh":
              predicted_tags.append([label_to_int_id[tag] for tag in prediction])
          else:
              new_prediction = []
              for x in prediction:
                  new_prediction.append(x)


              predicted_tags.append(list(label_to_int_id[label_to_tags_swapped[tag]] for tag in new_prediction))

      return predicted_tags

**defining some important dictionary**

In [35]:
label_to_tags = {'LABEL_0':'O', 'LABEL_1':'B-PER', 'LABEL_2':'I-PER', 'LABEL_3':'B-ORG', 'LABEL_4':'I-ORG', 'LABEL_5':'B-LOC', 'LABEL_6':'I-LOC','LABEL_7':'B-MISC','LABEL_8':'I-MISC'}
label_to_int_id = {'LABEL_0':0, 'LABEL_1':1, 'LABEL_2':2, 'LABEL_3':3, 'LABEL_4':4, 'LABEL_5':5, 'LABEL_6':6}
int_id_to_label = {0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4', 5: 'LABEL_5', 6: 'LABEL_6',7:'LABEL_7',8:'LABEL_8'}
label_to_tags_swapped = {value: key for key, value in label_to_tags.items()}
# label_to_tags_swapped


**setting paths for saved model**

In [29]:
Indic_Ner_3epoch_batch_8 = "/content/drive/My Drive/Indic_Ner_3epoch_batch_8"
Indic_BERT_3epoch_batch_8 = "/content/drive/My Drive/IndicBERT-3epoch_batch_16"
Indic_Ner_3epoch_batch_16 = "/content/drive/My Drive/Indic_Ner_3epoch_batch_16"
Indic_BERT_3epoch_batch_16 = "/content/drive/My Drive/Indic_BERT_3epoch_batch_8"
Indic_BERT_1lakh = "/content/drive/My Drive/Indic_BERT_1lakh"
Indic_BERT_50k ="/content/drive/My Drive/Indic_BERT_50k"
filtered_Indic_BERT = "/content/drive/My Drive/filtered_Indic_BERT_1lakh"
filtrered_Indic_NEr = "/content/drive/My Drive/filtrered_Indic_NEr_1lakh"

all_models_path = {
    0: Indic_Ner_3epoch_batch_8,
    1: Indic_BERT_3epoch_batch_8,
    2: Indic_Ner_3epoch_batch_16,
    3: Indic_BERT_3epoch_batch_16,
    4: Indic_BERT_1lakh,
    5: Indic_BERT_50k,
    6: filtered_Indic_BERT,
    7: filtrered_Indic_NEr
}

all_models_names = {
    0: "Indic_Ner_3epoch_batch_8",
    1: "Indic_BERT_3epoch_batch_8",
    2: "Indic_Ner_3epoch_batch_16",
    3: "Indic_BERT_3epoch_batch_16",
    4: "Indic_BERT_1lakh",
    5: "Indic_BERT_50k",
    6: "filtered_Indic_BERT",
    7: "filtrered_Indic_NEr"
}


**Load finetuned  Model**

In [24]:
# load saved fine-tuned model
import numpy as np
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
def load_model(path):
    tokenizer = AutoTokenizer.from_pretrained(path)
    model = AutoModelForTokenClassification.from_pretrained(path)
    return model,tokenizer


**prediction for one sentence exapmle**

In [25]:
def one_Sentence(path,sentence):
      model,tokenizer = load_model(path)
      prediction = get_predictions(sentence=sentence,
                                    tokenizer=tokenizer,
                                    model=model
                                    )
      last_part = path.split("/")[-1]
      if last_part != "Indic_Ner_3epoch_batch_8" and last_part!= "Indic_Ner_3epoch_batch_16" and last_part!="filtrered_Indic_NEr_1lakh":
          for index in range(len(sentence.split(' '))):
              print( sentence.split(' ')[index] + '\t' + label_to_tags[prediction[index]] )
      else:
          for index in range(len(sentence.split(' '))):
              print( sentence.split(' ')[index] + '\t' + prediction[index] )



## one sentence prediction of all model

In [15]:
sentence = "विश्लेषकों की सिफारिश के मुताबिक, इंद्रप्रस्थ गैस और पीएनबी हाउसिंग फाइनैंस भी अच्छे विकल्प हो सकते हैं।"
for path in all_models_path.values():
    last_part = path.split("/")[-1]
    print(f"\n\nModel : {str(last_part)}\n")
    one_Sentence(path,sentence)







Model : Indic_Ner_3epoch_batch_8

विश्लेषकों	B-LOC
की	B-LOC
सिफारिश	B-LOC
के	B-LOC
मुताबिक,	B-LOC
इंद्रप्रस्थ	B-LOC
गैस	I-LOC
और	B-LOC
पीएनबी	B-LOC
हाउसिंग	I-LOC
फाइनैंस	B-LOC
भी	B-LOC
अच्छे	B-LOC
विकल्प	B-LOC
हो	B-LOC
सकते	B-LOC
हैं।	B-LOC


Model : IndicBERT-3epoch_batch_16

विश्लेषकों	O
की	O
सिफारिश	O
के	O
मुताबिक,	O
इंद्रप्रस्थ	O
गैस	O
और	O
पीएनबी	O
हाउसिंग	O
फाइनैंस	O
भी	O
अच्छे	O
विकल्प	O
हो	O
सकते	O
हैं।	O


Model : Indic_Ner_3epoch_batch_16

विश्लेषकों	B-LOC
की	B-LOC
सिफारिश	B-LOC
के	B-LOC
मुताबिक,	B-LOC
इंद्रप्रस्थ	B-LOC
गैस	I-LOC
और	B-LOC
पीएनबी	B-LOC
हाउसिंग	I-LOC
फाइनैंस	I-ORG
भी	B-LOC
अच्छे	B-LOC
विकल्प	B-LOC
हो	B-LOC
सकते	B-LOC
हैं।	B-LOC


Model : Indic_BERT_3epoch_batch_8

विश्लेषकों	O
की	O
सिफारिश	O
के	O
मुताबिक,	O
इंद्रप्रस्थ	O
गैस	O
और	O
पीएनबी	O
हाउसिंग	O
फाइनैंस	O
भी	O
अच्छे	O
विकल्प	O
हो	O
सकते	O
हैं।	O


Model : Indic_BERT_1lakh

विश्लेषकों	O
की	O
सिफारिश	O
के	O
मुताबिक,	O
इंद्रप्रस्थ	B-ORG
गैस	I-ORG
और	O
पीएनबी	B-ORG
हाउसिंग	O
फाइनैंस	O
भी	O
अच्छे	O
विकल्प	O
ह

**finding precision , recall, F1 score and Macro F1 score**

##On test data

In [86]:
def each_model_score(data,true_tags,flag=0):
    for path in all_models_path.values():
        last_part = path.split("/")[-1]
        print(f"\n\nModel : {str(last_part)}\n")
        predicted_tags = prediction_test(path,data,flag)
        if last_part!="my_model_filtrered_Indic_NEr":
            calculate_scores(true_tags,predicted_tags)

In [17]:
test_data = raw_datasets['test']['tokens']
test_true_tags = raw_datasets['test']['ner_tags']
len(test_data) , len(test_true_tags)

(867, 867)

#Test data predictions

In [ ]:
each_model_score(test_data,test_true_tags)



Model : filtrered_Indic_NEr_1lakh

<class 'list'> <class 'list'>
Class O:
  Precision: 0.10666666666666667
  Recall: 0.0010094000378525014
  F1 Score: 0.001999875007812012
Class B-PER:
  Precision: 0.026722925457102673
  Recall: 0.025991792065663474
  F1 Score: 0.02635228848821082
Class I-PER:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
Class B-ORG:
  Precision: 0.013054830287206266
  Recall: 0.02032520325203252
  F1 Score: 0.01589825119236884
Class I-ORG:
  Precision: 0.6823308270676691
  Recall: 0.7756410256410257
  F1 Score: 0.726
Class B-LOC:
  Precision: 0.0029249062122464553
  Recall: 0.07718120805369127
  F1 Score: 0.00563621883232249
Class I-LOC:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
----------------------------
Macro F1 Score: 0.1108409476458163


Model : filtrered_Indic_NEr_1lakh

<class 'list'> <class 'list'>
Class O:
  Precision: 0.8310523348198268
  Recall: 0.997907305964178
  F1 Score: 0.9068687772681507
Class B-PER:
  Precision: 0.0
  Recall: 0.0
  F1 Score

## On Validartion data

In [ ]:
validation_data = raw_datasets['validation'].shuffle(seed=42).select(range(1000))['tokens'][:1000]
validation_true_tags = raw_datasets['validation']['ner_tags'][:1000]

In [ ]:
each_model_score(validation_data,validation_true_tags)



Model : Indic_Ner_3epoch_batch_8

<class 'list'> <class 'list'>
Class O:
  Precision: 0.875
  Recall: 0.017766497461928935
  F1 Score: 0.03482587064676617
Class B-PER:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
Class I-PER:
  Precision: 0.05263157894736842
  Recall: 0.058823529411764705
  F1 Score: 0.05555555555555555
Class B-ORG:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
Class I-ORG:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
Class B-LOC:
  Precision: 0.03768844221105527
  Recall: 0.8333333333333334
  F1 Score: 0.07211538461538461
Class I-LOC:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
----------------------------
Macro F1 Score: 0.02321383011681519


Model : IndicBERT-3epoch_batch_16

<class 'list'> <class 'list'>
Class O:
  Precision: 0.8330341113105925
  Recall: 0.9978494623655914
  F1 Score: 0.908023483365949
Class B-PER:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class I-PER:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class B-ORG:
  Precision: 0.0
  Rec

**on 20 percent test data**

## On Train data

In [18]:
train_dataset = raw_datasets['train']
_20k_train_data = train_dataset.shuffle(seed=42).select(range(1000))
len(_20k_train_data)
train_data = _20k_train_data['tokens']
train_tru_tags = _20k_train_data['ner_tags']

In [19]:
each_model_score(_20k_train_data,train_tru_tags)



Model : Indic_Ner_3epoch_batch_8

<class 'list'> <class 'list'>
Class O:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class B-PER:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class I-PER:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class B-ORG:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class B-LOC:
  Precision: 0.10714285714285714
  Recall: 1.0
  F1 Score: 0.19354838709677416
----------------------------
Macro F1 Score: 0.038709677419354833


Model : IndicBERT-3epoch_batch_16

<class 'list'> <class 'list'>
Class O:
  Precision: 0.55
  Recall: 1.0
  F1 Score: 0.7096774193548387
Class B-PER:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class I-PER:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class B-ORG:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class I-ORG:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class B-LOC:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
----------------------------
Macro F1 Score: 0.11827956989247312


Model : Indic_Ner_3epoch_batch_16


**check for one selected model**

In [ ]:
model_name = int(input("select Model\n0: Indic_Ner_3epoch_batch_8\n1: Indic_BERT_3epoch_batch_8\n2: Indic_Ner_3epoch_batch_16\n3: Indic_BERT_3epoch_batch_16\n4: Indic_BERT_1lakh\n5: Indic_BERT_50k\n6: filtered_Indic_BERT\n7: filtrered_Indic_NEr\n"))
print(f"You select {all_models_names[model_name]}")

select Model
0: Indic_Ner_3epoch_batch_8
1: Indic_BERT_3epoch_batch_8
2: Indic_Ner_3epoch_batch_16
3: Indic_BERT_3epoch_batch_16
4: Indic_BERT_1lakh
5: Indic_BERT_50k
6: filtered_Indic_BERT
7: filtrered_Indic_NEr
0
You select Indic_Ner_3epoch_batch_8


In [ ]:
path = all_models_path[model_name]

predicted_tags = prediction_test(path)


In [ ]:
true_tags = test_true_tags

In [ ]:
type(predicted_tags[0]), type(true_tags)

(list, list)

In [ ]:
calculate_scores(true_tags,predicted_tags)

<class 'list'> <class 'list'>
Class O:
  Precision: 0.2702702702702703
  Recall: 0.003154375118289067
  F1 Score: 0.006235969069593414
Class B-PER:
  Precision: 0.040268456375838924
  Recall: 0.04103967168262654
  F1 Score: 0.04065040650406504
Class I-PER:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
Class B-ORG:
  Precision: 0.01453104359313078
  Recall: 0.022357723577235773
  F1 Score: 0.017614091273018415
Class I-ORG:
  Precision: 0.6458333333333334
  Recall: 0.7948717948717948
  F1 Score: 0.7126436781609194
Class B-LOC:
  Precision: 0.003601054594559835
  Recall: 0.09395973154362416
  F1 Score: 0.006936272991887037
Class I-LOC:
  Precision: 0.0017482517482517483
  Recall: 0.005208333333333333
  F1 Score: 0.002617801047120419
----------------------------
Macro F1 Score: 0.11238545986380054


In [ ]:
sentence = "विश्लेषकों की सिफारिश के मुताबिक, इंद्रप्रस्थ गैस और पीएनबी हाउसिंग फाइनैंस भी अच्छे विकल्प हो सकते हैं।"
one_Sentence(path,sentence)

विश्लेषकों	B-LOC
की	B-LOC
सिफारिश	B-LOC
के	B-LOC
मुताबिक,	B-LOC
इंद्रप्रस्थ	B-LOC
गैस	I-LOC
और	B-LOC
पीएनबी	B-LOC
हाउसिंग	I-LOC
फाइनैंस	B-LOC
भी	B-LOC
अच्छे	B-LOC
विकल्प	B-LOC
हो	B-LOC
सकते	B-LOC
हैं।	B-LOC


#Answer 4

In [7]:
tags_to_int_label = {'O':0,'B-PER':1,'I-PER':2,'B-ORG':3,'I-ORG':4,'B-LOC':5,'I-LOC':6,'B-MISC':7,'I-MISC':8}

In [16]:
import ast
def extract_tags_from_file(path):
    tags = []
    with open(path, 'r', encoding='UTF-8') as file:
        lines = file.readlines()

        for i, line in enumerate(lines, start=1):
            if i % 4 == 2:
                tag = ast.literal_eval(line.strip())
                tags.append([tags_to_int_label[ele] for ele in tag])
    return tags




def extract_Sentences(path):
    sentences = []
    with open(path, 'r', encoding='UTF-8') as file:
        lines = file.readlines()

        for i, line in enumerate(lines, start=0):
            if i % 4 == 0:
              sentences.append(line)


    return sentences


In [8]:
chatgpt_tags = extract_tags_from_file("/content/drive/MyDrive/NER_Tags/CS698_chatgpt_NEr_hi.txt")
manual_tags = extract_tags_from_file("/content/drive/MyDrive/NER_Tags/CS698_manual_NEr_hi.txt")

## Results on manual tag and chatgpt tags

In [ ]:
calculate_scores(chatgpt_tags,manual_tags)

<class 'list'> <class 'list'>
Class O:
  Precision: 0.9922928709055877
  Recall: 0.9346642468239564
  F1 Score: 0.9626168224299065
Class B-PER:
  Precision: 0.55
  Recall: 0.6111111111111112
  F1 Score: 0.5789473684210527
Class I-PER:
  Precision: 0.4117647058823529
  Recall: 0.7777777777777778
  F1 Score: 0.5384615384615384
Class B-ORG:
  Precision: 0.3333333333333333
  Recall: 0.75
  F1 Score: 0.46153846153846156
Class I-ORG:
  Precision: 0.2857142857142857
  Recall: 1.0
  F1 Score: 0.4444444444444445
Class B-LOC:
  Precision: 0.14285714285714285
  Recall: 0.5
  F1 Score: 0.22222222222222224
Class B-MISC:
  Precision: 0.16666666666666666
  Recall: 1.0
  F1 Score: 0.2857142857142857
Class I-MISC:
  Precision: 0.3333333333333333
  Recall: 1.0
  F1 Score: 0.5
----------------------------
Macro F1 Score: 0.49924314290398897


In [17]:
sentences = extract_Sentences("/content/drive/MyDrive/NER_Tags/CS698_manual_NEr_hi.txt")

In [88]:
## Remove numbering and newline characters from each sentence
sentences_cleaned = [sentence.replace('\n', '').split('. ')[1] for sentence in sentences]



In [71]:
import re

In [72]:
pattern = r'([.,?\(\)\।\‘’\'\"])'

In [73]:
for i in range(len(sentences_cleaned)):
  sentences_cleaned[i] = re.sub(pattern,r' \1 ',sentences_cleaned[i])
  sentences_cleaned[i] = re.sub('\xa0', ' ',sentences_cleaned[i])
  sentences_cleaned[i] = re.sub('…', ' . . . ',sentences_cleaned[i])
  sentences_cleaned[i].strip()

## Calculations on 25 sentences

In [87]:
each_model_score(sentences_cleaned,manual_tags,1)



Model : Indic_Ner_3epoch_batch_8

<class 'list'> <class 'list'>
Class O:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class B-PER:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
Class I-PER:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
Class B-ORG:
  Precision: 0.07142857142857142
  Recall: 0.14285714285714285
  F1 Score: 0.09523809523809523
Class I-ORG:
  Precision: 1.0
  Recall: 0.5714285714285714
  F1 Score: 0.7272727272727273
Class B-LOC:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
Class B-MISC:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class I-MISC:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
----------------------------
Macro F1 Score: 0.10281385281385282


Model : IndicBERT-3epoch_batch_16

<class 'list'> <class 'list'>
Class O:
  Precision: 0.8863636363636364
  Recall: 1.0
  F1 Score: 0.9397590361445783
Class B-PER:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class I-PER:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class B-ORG:
  Precision: 1.0
  Recall: 